In [5]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Loss: 1.790 — Avg: 1.805 — GPU Mem: 13512 MB:   2%|▏         | 1120/64000 [04:58<4:38:52,  3.76it/s]
Cinnamon Roll Pull Apart Bread

8 oz cream cheese
2 tablespoons sugar
1 teaspoon cinnamon
8 teaspoon cinnamon rolls


1. In a bowl, combine cream cheese, sugar, and cinnamon.
2. On a floured surface, flatten each cinnamon roll and spread cream cheese mixture on top.
3. Cut flattened roll in half and place one half on top of the other. Repeat for the rest of the cinnamon rolls.
4. Line a loaf pan with parchment paper. Lay the pan vertically on its side, and stack cinnamon rolls cut side facing the bottom of the pan.
5. Cover the loaf pan loosely with foil.
6. Bake at 325˚F (160˚C) 70 minutes, removing the foil for the last 10 minutes.
7. Drizzle with the icing that comes with the cinnamon rolls.
8. Enjoy!



In [6]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe2.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [7]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)


In [8]:

file_name = "recipe2.csv"
ai.save("aitextgen.model")
train_tokenizer(file_name)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=2000, num_steps=64000, generate_every=1000, save_every=5000)
ai.generate(prompt="steak")

#Save the model
ai.save("aitextgen.model")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.
100%|██████████| 9922/9922 [00:00<00:00, 12520.17it/s]
pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: generating sample texts.                                               
 with the remaining water, and milk.                                                                
5. Pour the egg mixture into the bowl with a rubber spatula.
6. Line a large baking tray with the pizza dough with parchment paper.
7. Spread the pizza dough into
2,000 steps reached: generating sample texts.                                               
er-dried biscuits on the top of the buns, then top with the remaining cheese.                       
15. Broil the bun for 6-8 minutes, or until the edges are golden brown.
16. Carefully remove the b
3,000 steps reached: generating sample texts.                                               
.                                                                                                   
2. Make the crust: In a large bowl, whisk together the flour, baking powder, salt, and salt.
3. In a medium bowl, combine the butter, sugar, and sugar, and 

In [ ]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen
ai = aitextgen(model_folder="trained_model",
                tokenizer_file="aitextgen.tokenizer.json")

ai.load("aitextgen.model")             

NameError: name '_C' is not defined

In [ ]:
ai.save("aitextgen.model")